# Project Lab #1: Part 2

## Writing more core functions, and a script to compute and plot a full spacecraft trajectory.

___ 

### Part 2A: Use the section below and the pre-reading to write the following functions: 
- `grav_acc` 
- `checkinit`
- `sc_vel_pos_change`
- `get_traj`  


### Part 2B:  More planning:

Using the pseudocode you wrote in Part 1, look and decide:  
1.  Where in your code will you add the call to `checkinit`? 
2.  What bits of code will go into `sc_vel_pos_change` and how/where will this function be called? 
3.  After doing 1 and 2: you'll notice that almost all of your pseudo code, except the definitions of initial conditions, time step and total trajectory duration will go into `get_traj`.  


### Part 2C: Run a simulation:
- Call your function `get_traj` with the following values for the various parameters needed. In your code, remember to convert everything to SI units (kg, m, s) before doing ANY calculations:  
> - Mercury mass, $M_\mathrm{p} = 3.3 \times 10^{23}$ kg  
> - Mercury radius, $R_\mathrm{p} = 2440$ km.  Needed for spacecraft altitude determinations and plotting.
> - Total time, $t_\mathrm{f}$ = 40 minutes   
> - Time increment, $\Delta t = 60$ seconds  
> - Initial position, $s_\mathrm{x 0}=-3050 \mathrm{~km}, s_\mathrm{y 0}=-3 R_{p}$    
> - Initial velocity, in positive y-direction with magnitude $7$ km s$^{−1}$  

These parameters simulate the flybys of Mercury by the MESSENGER spacecraft in 2008 and 2009 to alter the trajectory such that the spacecraft could be put into orbit around the planet in 2011.


### Part 2D: Check it's all working:

Make 2 figures (they should look like the ones in this notebook that are included after the function descriptions below): 

- **Figure 1** should plot the spacecraft trajectory in the x, y plane, along with a circle to denote the cross-section of Mercury. 
    
- **Figure 2** should contain 3 subplots (3 rows, 1 column):   
> - **Subplot 1**: (top) plot $a_x$, $a_y$, $|a|$ versus time (where $|a|$ is the magnitude of the acceleration vector).    
> - **Subplot 2**: (middle) plot the speed $v$ versus time.   
> - **Subplot 3**: (bottom) plot spacecraft altitude (height above planet’s surface) versus time.


___

## Function Definitions & Usage

Your functions must perform as indicated in the descriptions below.

<div class="alert alert-block alert-warning">
Note that for the all the new functions for Part 2 the functions <b>checkinit</b> , <b>sc_vel_pos_change</b> and <b>get_traj</b>, you should pass vector quantities (acceleration, velocity, position, change in velocity, change in position) in and out of functions as individual components separately (not as a list or array).   So e.g. the usage for <b>checkinit</b> should be


`checkinit(s_x0, s_y0, v_x0, v_y0, planet_radius)`.
       
</div>

___ 

### function 1: `grav_acc` 

- Use your working function from Week 10.

<br> 

___ 

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

def grav_acc(s_x, s_y, planet_mass):
    '''
    takes in x,y components of position vector from the planet and mass of planet
    finds acceleration components

    input: s_x, s_y, planet_mass
    output: a_x (m/s), a_y (m/s)
    '''
    s = math.sqrt(s_x**2 + s_y**2)
    #formula for a is GMp/s^2
    big_G = 6.67e-11
    a = (big_G*planet_mass)/(s**2)

    sin_beta = -(s_x)/s
    cos_beta = -(s_y)/s

    a_x = a * sin_beta
    a_y = a * cos_beta

    return(a_x, a_y)

### function 2: `checkinit`


**Code Description:**  

The body of `checkinit` should implement the following checks: 
- the starting position of the spacecraft is above the planetary surface:   $|s| > R_p$, where $|s|$ is the magnitude of the position vector. 
- the starting velocity is in the positive y-direction (spacecraft flying toward the planet not away from it!).

If either of these conditions are not met, the program should terminate with a helpful error message and by raising a `ValueError`.   

**Usage:**
```python
checkinit(s_x0, s_y0, v_x0, v_y0, planet_radius)
```

**Inputs:**
- `s_x0, s_y0`: spacecraft initial x and y components of position in m. 
- `v_x0, v_y0`: spacecraft initial x and y components of velocity in m/s.
- `planet_radius`: the radius of the planet $R_\mathrm{p}$, in m.

**Outputs:** None.

<br> 

**Note:**
The syntax to raise a `ValueError` is simply `raise ValueError`.

<br>

___

In [3]:
def checkinit(s_x0, s_y0, v_x0, v_y0, planet_radius):
    s = math.sqrt(s_x0**2 + s_y0**2)
    if np.abs(s) <= planet_radius:
        raise ValueError('lmfao loser')
    elif v_y0 < 0:
        raise ValueError('lmao')
    else:
        pass

### function 3: `sc_vel_pos_change`

**Code Description:**

- Implement **Equations (10) - (13)** in the lab reading to compute the instantaneous (i.e. just at a single time-step) **change** in position and velocity of the spacecraft. 

**Usage:** 

```python
(ds_x, ds_y, dv_x, dv_y) = sc_vel_pos_change(a_x, a_y, v_x, v_y, time_step)
```

**Inputs:** 
- `a_x, a_y`: instantaneous x, y components of acceleration in m/s^2
- `v_x, v_y`: instantaneous x,y components of velocity in m/s.
- `time_step`: the time increment, $\Delta t$ in s.

**Outputs:**
- `ds_x, ds_y`: change in position vector x- and y- components in m 
- `dv_x, dv_y`: change in velocity vector x- and y- components in m/s

<br> 


___

In [4]:
def sc_vel_pos_change(a_x, a_y, v_x, v_y, time_step): 
    dv_x = a_x * time_step
    dv_y = a_y * time_step
    
    ds_x = v_x * time_step + 0.5*a_x*(time_step)**2
    ds_y = v_y * time_step + 0.5*a_y*(time_step)**2
    
    return ds_x, ds_y, dv_x, dv_y

### function 4: `get_traj`

**Code Description:**

The body of `get_traj` should implement your pseudo-code to compute a full spacecraft trajectory.  It will call `grav_acc`, `sc_vel_pos_change`, and `checkinit`.

**Usage:**

```python
time, acc, vel, pos = get_traj(s_x0, s_y0, v_x0, v_y0, time_step, total_time, planet_mass, planet_radius)
```

**Inputs:**

- `s_x0, s_y0`: spacecraft's initial position x-, y- components in m.
- `v_x0, v_y0`: spacecraft's initial velocity x-, y- components in m/s.
- `time_step`: floating-point or integer time increment, $\Delta t$ in seconds, at which spacecraft acceleration, velocity, positions are updated.
- `total_time`: floating-point or integer total-time value for the integration in seconds.
- `planet_mass`: floating-point or integer mass of fly-by target planet $M_\mathrm{p}$ (default = 1 Mercury mass) in kg. 
- `planet_radius`: floating-point or integer radius of fly-by target planet $M_\mathrm{p}$ (default = 1 Mercury radius) in m. 

<div class="alert alert-block alert-warning">
<b>ESSENTIAL:</b>  Any positional arguments should be in the order shown in usage for ALL functions.    
</div>  

 
**Outputs:**

- `time`: an array of times with shape `(nt, )`.   
- `acc`: `(acc_x, acc_y`), an array of spacecraft accelerations with shape `(nt, 2)`. 
- `vel`: `(vel_x, vel_y)`, an array of spacecraft velocities with shape `(nt, 2)` . 
- `pos`: `(pos_x, pos_y)`, an array of spacecraft positions with shape `(nt, 2)`.


<div class="alert alert-block alert-warning">

**ESSENTIAL:**
- `acc[0,:]`, `vel[0,:]` and `pos[0,:]` should contain the spacecraft acceleration, velocity and position at time `t=0`.
- `acc[nt,:]`, `vel[nt,:]` and `pos[nt,:]` should contain the spacecraft acceleration, velocity and position at time `t = total_time`.    

</div>  

In [ ]:
def get_traj(s_x0, s_y0, v_x0, v_y0, time_step, total_time, planet_mass, planet_radius):
    total_steps = int(total_time/time_step) + 1
    
    time = np.linspace(0, total_time, total_steps)
    
    acc = np.ones((time.size, 2))*np.nan
    vel = np.ones((time.size, 2))*np.nan
    pos = np.ones((time.size, 2))*np.nan
    
    checkinit(s_x0, s_y0, v_x0, v_y0, planet_radius)
        
    pos[0, 0] = s_x0
    pos[0, 1] = s_y0
    vel[0, 0] = v_x0
    vel[0, 1] = v_y0
    acc[0, 0], acc[0, 1] = grav_acc(pos[0, 0], pos[0, 1], planet_mass)

    for i in range(1, len(time)):

        a_x, a_y = grav_acc(pos[i-1, 0], pos[i-1, 1], planet_mass)
        
        acc[i, 0] = a_x 
        acc[i, 1] = a_y 
        
        ds_x, ds_y, dv_x, dv_y = sc_vel_pos_change(a_x, a_y, vel[i-1, 0], vel[i-1, 1], time_step)
        
        vel[i, 0] = vel[i-1, 0] + dv_x
        vel[i, 1] = vel[i-1, 1] + dv_y
        pos[i, 0] = pos[i-1, 0] + ds_x
        pos[i, 1] = pos[i-1, 1] + ds_y
        
    return time, acc, vel, pos

In [ ]:
v_x0 = 0 #(m/s)
v_y0 = 7000 #(m/s)
time_step = 60 
total_time = 40 * 60 #(s)
planet_mass = 3.3e23 #(kg)
planet_radius = 2440*1000 #(m)
s_x0 = -3050*1000 #(m)
s_y0 = -3*planet_radius #(m)


time, acc, vel, pos = get_traj(s_x0, s_y0, v_x0, v_y0, time_step, total_time, planet_mass, planet_radius)
plt.plot(pos[: , 0]/1000, pos[: , 1]/1000)
plt.xlim(-9999, 9999)
plt.ylim(-9999, 9999)
plt.scatter(0, 0, marker = "o", s = 2440 * 2)
plt.grid(True, alpha=0.5, which='both', linewidth=0.5, axis='both')
plt.gca().set_aspect('equal', adjustable='box')


In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(12,6))

acc_magnitude = np.linalg.norm(acc, axis=1)

ax[0].plot(np.arange(time.size), acc[:, 0], label='a_x') 
ax[0].plot(np.arange(time.size), acc[:, 1], label='a_y') #it's a horror
ax[0].plot(np.arange(time.size), acc_magnitude, label='|a|') 

ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Acceleration (m/s^2)')
ax[0].set_title('Acceleration due to Gravitational Force')
ax[0].legend()
ax[0].grid()

speed = np.linalg.norm(vel, axis=1)
ax[1].plot(np.arange(time.size), speed)
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Velocity (m/s)')
ax[1].set_title('Spacecraft Speed')
ax[1].grid()

altitude = np.linalg.norm(pos, axis=1) - planet_radius
closest_approach_altitude = np.min(altitude)
ax[2].plot(np.arange(time.size), altitude)
ax[2].set_xlabel('Time (s)')
ax[2].set_ylabel('Altitude (m)')
ax[2].set_title('Spacecraft Altitude')
ax[2].text(len(time)/2, 6e6, f'Closest Approach Altitude: {closest_approach_altitude:.1f} m', va='center', ha='center')

fig.tight_layout()

___

## Figures to check Part 2D:

<div class="alert alert-block alert-info">
<b>Tip:</b> So that you can check your work at this point, your figures should look like Figures 1 and 2 below. A useful diagnostic to print to the screen is the closest approach altitude – you should get 365 km when rounded to the nearest integer.  
</div>


**Figure 1:**

<img src="./wk11_fig1.png" width="500">

**Figure 2:**

<img src="./wk11_fig2.png" width="800">

___

## Part 2 SUBMISSION - OPTIONAL, DUE FRIDAY November 17 (Week 11), 4 pm on Canvas.   

**Remember:** Suppress any extra `print()` statements that aren't necessary in your notebook with the code below AND in the notebook that produces the PDF.

**TO SUBMIT:**
1.  A single Jupyter notebook with ONLY 2 cells with contents EXACTLY as follows
> - Cell 1: a Markdown cell with your names and student numbers
> - Cell 2: A code cell with import statements and the four functions `grav_acc()`, `checkinit()`, `sc_vel_pos_change()` and `get_traj()`. 

    
2.  A `.pdf` file of your full Jupyter Notebook which includes your output for Figures 1 and 2. 

Note that this submission includes the function you wrote for Part 1.  We will repeat our tests from Part 1 on this function so if you corrected/updated it this will now be reflected in those autogrades.  